**My head is spinning from all the other amazing methods to work through best approach and features. Just so I can say I made a submittal on a competition, I'll send this one in. It may take awhile to read and understand the numerous ways to solve the problem. Thanks to everyone for creating/sharing their work!**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



**Load up the data, set target.**

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv', index_col = 'id')
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv', index_col = 'id')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv', index_col = 'id')
target = train.pop('target')

One-hot encode seems reasonable

In [ ]:
train_test_combined = pd.concat([train, test])
cat_cols = [feature for feature in train.columns if 'cat' in feature]
dummies = pd.get_dummies(train_test_combined, columns=cat_cols, drop_first=True)

# Create new train and test sets with one-hot encodings
train_ohe = dummies.iloc[:train.shape[0], :]
test_ohe = dummies.iloc[train.shape[0]:, :]


In [ ]:
display(train_ohe.head())

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_ohe, target, test_size=0.2, random_state=0)

# OPTUNA
Credit to Ernee Kozyreff (https://www.kaggle.com/ekozyreff) for enlightening me on optuna in his notebook (https://www.kaggle.com/ekozyreff/tps-2021-02-xgboost-optuna-gpu) ...really cool - thx!

In [ ]:
import optuna
def objective(trial):
    params = {
        'random_state': 42,
        'n_estimators': trial.suggest_categorical('n_estimators', [100]),
        'max_depth': trial.suggest_int('max_depth', 10, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.01),
        'subsample': trial.suggest_categorical('subsample', [0.9, 1.0])
    }
    model = GradientBoostingRegressor(**params) 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    
    return rmse

In [ ]:
%%time
study = optuna.create_study(direction='minimize',sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=10)
print('Number of finished trials:', len(study.trials))
print('Best parameters:', study.best_trial.params)
print('Best RMSE:', study.best_trial.value)

**By running optuna method, I used the reported, optimal values and set them accordingly.**


In [ ]:
#Set numbers manually from optuna tests for submittal.
#'n_estimators': 100, 'max_depth': 13, 'learning_rate': 0.009556428757689247, 'subsample': 0.9}


In [ ]:
model = GradientBoostingRegressor(
            max_depth= study.best_trial.params['max_depth'],
            subsample = study.best_trial.params['subsample'],
            n_estimators= study.best_trial.params['n_estimators'],
            learning_rate= float(study.best_trial.params['learning_rate'])
        )
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(test_ohe)
pred_df = pd.DataFrame(test_ohe.index)
pred_df['target'] = np.zeros(len(test_ohe))
pred_df.loc[:len(test_ohe),'target'] = predictions

In [ ]:
pred_df.to_csv('submission.csv',index=False)